In [6]:
import os
import sys
import time
import json
from datetime import datetime

import requests
from fastapi.responses import JSONResponse
from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import load_dotenv


load_dotenv(dotenv_path='/home/psor/testgit/test_project/get_stocks/.env')

STOCK_API_KEY = os.getenv('STOCK_API_KEY')
PROJECT_ID = os.environ.get('PROJECT_ID')  
# No need to explicitly provide credentials when running on Cloud Run. The BigQuery client will automatically use the service account associated with the Cloud Run service.
credentials = service_account.Credentials.from_service_account_file(
    "/mnt/c/Users/m_was/Downloads/tomastestproject-433206-48a55703dec2.json")
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

table_id = f"{PROJECT_ID}.testdb_1.raw_stock_data"

def fetch_and_upload_raw_stock_data(stock_symbol: str, API_KEY: str):
    try:
        # Construct the API URL
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_symbol}&apikey={STOCK_API_KEY}"
        response = requests.get(url=url)   
        response.raise_for_status()
        data = response.json()
        fetch_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Prepare data for insertion
        rows_to_insert = [
            {
                "stock_symbol": stock_symbol,
                "raw_data": json.dumps(data),
                "fetch_date": fetch_date  
            }
        ]
        return data
        errors = client.insert_rows_json(table_id, rows_to_insert)

        if errors:
            print(f"Encountered errors while inserting rows: {errors}")
        else:
            print("Rows successfully inserted.")
        
    except Exception as e:
        print(f"An error occurred: {e}")


#if __name__ == "__main__":


In [16]:
import os
import json
import requests
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import load_dotenv

load_dotenv()

# Load environment variables
PROJECT_ID = os.getenv('PROJECT_ID')
credentials = service_account.Credentials.from_service_account_file(
    "/mnt/c/Users/m_was/Downloads/tomastestproject-433206-48a55703dec2.json")
client = bigquery.Client(credentials=credentials, project=PROJECT_ID)

# BigQuery table ID
raw_data_table_id = f"{PROJECT_ID}.testdb_1.raw_stock_data"

def fetch_and_upload_raw_stock_data(stock_symbol: str, API_KEY: str):
    try:
        # Construct the API URL
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_symbol}&apikey={API_KEY}"
        response = requests.get(url=url)
        response.raise_for_status()
        data = response.json()

        # Extract the "Time Series (Daily)" part
        time_series_daily = data['Time Series (Daily)']

          # Extract only the first dictionary from "Time Series (Daily)"
        latest_date = sorted(time_series_daily.keys())[-1]
        latest_data = {latest_date: time_series_daily[latest_date]}

        # Prepare the row to insert
        row_to_insert = {
            "stock_symbol": stock_symbol,
            "raw_data": json.dumps(latest_data),
            "fetch_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

        


        # Insert the row into BigQuery
        errors = client.insert_rows_json(raw_data_table_id, [row_to_insert])
        if errors:
            print(f"Encountered errors while inserting rows: {errors}")
        else:
            print("Latest day's raw data successfully inserted into BigQuery.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
if __name__ == "__main__":
    STOCK_API_KEY = "your_api_key_here"
    fetch_and_upload_raw_stock_data("TSLA", STOCK_API_KEY)

Latest day's raw data successfully inserted into BigQuery.


In [12]:
fetch_and_upload_raw_stock_data("TSLA", STOCK_API_KEY)

An error occurred: 0
